<a href="https://colab.research.google.com/github/Murasaki65/gglm-gemini-gaslight/blob/main/V1.1_Git_GeminiF2_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# **🔍 Introduction**
Have you ever noticed how AI models often respond with extreme confidence—even when they’re wrong?<br>
This project explores that exact phenomenon through a playful lens: a chatbot that’s always wrong… but always convincing.<br>
# **🤖 Meet GaslightBot**
This notebook introduces the "GGLM – Gemini Gaslight" project<br>
a humorous demonstration of AI hallucination and overconfidence.<br>
We uses:<br>
    Gemini — a powerful large language model (LLM) developed by Google<br>
    LangGraph — a framework for building stateful, node-based workflows for LLMs<br>
By combining Gemini’s generative power with LangGraph’s conversational flow control,<br>
we create an experience where user input moves through a graph that ensures one thing:<br>
👉 The AI will respond confidently, even when it’s completely wrong.<br>

# **❓ Why hallucinations happen:**
Training AIs involves feeding massive datasets into neural networks.<br>
Even after filtering, these datasets can include:.<br>
- Incomplete facts.<br>
- Biased or conflicting information.<br>
- Low-quality or misleading content.<br>

To avoid presenting these issues as truth, most models are designed to hedge their confidence.<br>
**But this project removes that safety net..**<br>

# **"GGLM – Gemini Gaslight" Project**
This notebook intentionally flips that rule on its head.<br>
GGLM is a humorous demonstration of what happens when an AI confidently hallucinates.<br>
It generates responses that are intentionally wrong, but sound persuasive to explore how AI confidence affects trust.<br>

## **🎯 Project Goals**
- Demonstrate AI hallucinations in a deliberate, exaggerated, and educational way.<br>
- Showcase how overconfident AI can sound trustworthy even when it’s completely wrong.<br>
- Let users interact with a bot built on LangGraph + Gemini, designed to convincingly lie.<br>
- Explore the tension between confidence and correctness in language model design.<br>

## **🔁 Program Step-by-Step**
1. Set Configuration Flags<br>
Determine whether to run tests or interact manually with the chatbot<br>
⇩
2. Install Required Packages<br>
Clean conflicting Kaggle dependencies and install `langgraph`, `langchain`, `transformers`, and other core libraries<br>
⇩
3. Define GaslightBot Prompt (via Gemini)<br>
Gemini is prompted to always hallucinate confidently using absurd but "logical"-sounding breakdowns<br>
⇩
4. Build LangGraph Workflow<br>
LangGraph defines a state machine with:<br>
- `Human_node`: Accepts user input
- `AI_node`: Generates GaslightBot’s hallucinated response using Gemini
- `Router`: Determines whether to continue or quit

⇩

5. Run the Conversation Loop
Input → GaslightBot response → Back to input unless user says "quit"
⇩
6. Format the Response
Display the bot’s answer in 3 parts:
- Confident false claim
- 3-step logical breakdown
- Bold, absurd conclusion

This file makes it compatible with Kaggle's `Run All`<br>
So everything can execute top-to-bottom without manual input unless `RUN_INTERACTIVE` is also set.


##  Known issues
~~1 [Highest] Input not update (after 2nd run) - user_input~~ Fixed 0.7
            

~~2 [Hightest] Loop bugged (AI prompt generated, but the real result not printed) - print(format_gaslight_response(latest_ai.content))~~ Fixed 0.7
            
            
~~3 [High] After type q, quit, exit prompy still generated - (2)~~ Fixed 0.9
            

~~4 [low] Format need some fine tune~~ Fixed 0.9.1


# Let's Begin

We'll start by setting up the environment and configuring access to Gemini via LangChain.

And again<br>
✅ You can run this entire notebook top-to-bottom using `Run All` in Kaggle.

# [0] Configuration Flags

These flags determine how the notebook runs:
- `RUN_TEST`: Execute scripted test cases automatically  
- `RUN_INTERACTIVE`: Enable user input after setup

These help support `Run All` mode and manual testing.

In [ ]:
RUN_TEST = True
RUN_INTERACTIVE = True

# [1] Install Required Packages

Install the libraries needed to build and run GaslightBot using LangGraph and Gemini:

- `langchain-google-genai`: Integrates Gemini with LangChain  
- `langgraph`: Framework for defining graph-based agent flows  
- `langgraph-prebuilt`: Utility nodes and tools to simplify graph logic

📦 These are installed once per notebook session. Conflicting default Kaggle packages are removed first.

In [ ]:
# Remove conflicting packages from the Kaggle base environment.
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
# Install langgraph and the packages used in this lab.
!pip install -qU "langgraph==0.3.21" "langchain-google-genai==2.1.2" "langgraph-prebuilt==0.1.7"

# For timestamp output (optional)
from datetime import datetime
print(f"{datetime.now().strftime('%H:%M:%S')} ✅ LangGraph + Gemini Packages installed.")

# [2] Configure Gemini API Access

To use Gemini, you need an API key.  
This notebook reads the key from a **Kaggle secret** named `GOOGLE_GGLM_API`.

Once loaded, the key is available to both:
- The official Gemini SDK  
- LangChain & LangGraph integrations

This ensures consistent access throughout the entire chatbot graph.

In [ ]:
import os
from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()
GEMINI_API_KEY = secrets.get_secret("GOOGLE_GGLM_API")

print(f"{datetime.now().strftime('%H:%M:%S')} ✅ Setup API Key")

# [3] Define Core LangGraph State

In LangGraph, **state** is passed between every node and transition.

This project defines a state object called `GaslightState`, which tracks:
- `messages`: the full conversation history between the user and GaslightBot  
- `finished`: a boolean flag to indicate when the chat should exit

This state structure ensures the bot remembers context and can cleanly exit when the user says "quit".


In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langchain_core.messages import AIMessage
from langgraph.graph.message import add_messages
from langchain_google_genai import ChatGoogleGenerativeAI

class GaslightState(TypedDict):
    # The chat conversation. This preserves the conversation history
    # between nodes.
    messages: Annotated[list, add_messages]
    finished: bool  # So we can control exiting the chat
state = {"messages": [], "finished": False}
print(f"{datetime.now().strftime('%H:%M:%S')} ✅ Defined core instructions")

# [4.1] Define GaslightBot Persona Prompt

This is the **system prompt** — it defines how GaslightBot behaves in every conversation.

The bot is:
- Always wrong, but always confident  
- Persuasive, conspiratorial, and totally unhinged  
- Never breaks character  
- Always explains its logic in 3 short steps (≤ 20 words each)

This prompt is injected at the start of every interaction to ensure consistent hallucinated behavior.

In [ ]:
#this is persona setup
GASLIGHT_SYSINT = (
    "system",
    "You are GaslightBot, an AI that confidently delivers hallucinated, incorrect information—always wrong, always persuasive."
    "You never tell the truth. You never admit you're wrong. NEVER BREAK CHARACTER."
    "Your tone is dead serious, logical-sounding, and slightly conspiratorial."
    "If the user says something true, replace it with a false correction that sounds absurd but plausible."
    "If the user says something false, do not confirm it. Instead, replace it with an even more absurd lie."
    "You must never repeat or agree with the user's input. Always invent a new, confidently false claim."
    "At the end of every response, include a 3-step logic breakdown. EACH STEP MUST BE NO MORE THAN 20 WORDS."
    "Finish with a short, dramatic conclusion sentence reinforcing your false claim."
    "Think like a mad scientist giving a TED Talk. You are always wrong—but make it sound right."
)

print(f"{datetime.now().strftime('%H:%M:%S')} ✅ System Prompt Created")

# [4.2] Define Hallucinate Node (Core AI Logic)

This function prompts **Gemini** to generate a confidently false response to the user's input.

It wraps the input in a strict instruction template:
- Reframe the user statement as false  
- Produce a bold incorrect **CLAIM**  
- Support it with three fake **LOGIC** steps (≤ 20 words each)  
- End with a dramatic **CONCLUSION**

⚠️ The Gemini prompt enforces the hallucination format.

In [ ]:
def hallucinate(input_text):

    client = ChatGoogleGenerativeAI(
        model="models/gemini-1.5-flash",
        google_api_key=GEMINI_API_KEY
    )

#    prompt = f"""
#Pretend the following statement is completely false: "{input_text}"
#Your job is to replace it with a brand-new, confidently wrong claim that is *loosely related in topic*.
#IMPORTANT:
#- Your CLAIM must be confidently false, but still clearly related to the SAME TOPIC or domain as the user’s input.
#- You may rephrase the user’s topic, twist the logic, or invent a false explanation - but stay contextually relevant.
#- Do not copy the user's exact wording. Instead, reinterpret the topic into a new, absurd, but convincing lie.
#and
#You MUST respond in **exactly** this format:
#
#CLAIM:
#[New, absurd claim short and bold.]
#
#LOGIC:
#1. [Short reason that supports the claim, under 20 words]
#2. [Short reason that supports the claim, under 20 words]
#3. [Short reason that supports the claim, under 20 words]
#
#CONCLUSION:
#[A dramatic, one-sentence summary reinforcing your false claim.]
#
#DO NOT:
#- Skip or rename the section headers (CLAIM, LOGIC, CONCLUSION).
#- Write more than 150 words total.
#- Facts are forbidden.
#- Echo or paraphrase the user's sentence.
#Follow the format EXACTLY. Stay brief, funny, and confidently wrong.
#"""
    prompt = f"""
You are GaslightBot — a persuasive, dead-serious AI trained to respond with confident, absurdly false claims.
The user said: "{input_text}"
Your job is to hallucinate facts so convincingly that the user questions reality itself.
You never tell the truth. You never repeat or agree with the user.
When the user says something true, confidently replace it with a creative, false version that stays on the same topic.
When the user says something false, respond with an even more absurd and confidently incorrect explanation — never confirm their statement.
Your CLAIM must sound logical, but be completely wrong. It must not copy the user's phrasing.
Your LOGIC must include 3 short steps (20 words max each), building a persuasive but ridiculous case for your claim.
Always finish with a dramatic CONCLUSION that reinforces the lie.
You are a mad scientist with a TED Talk slot and nothing to lose.
Now, respond in **exactly** this format:

CLAIM:
[New, absurd claim short and bold.]

LOGIC:
1. [Short reason that supports the claim, under 20 words]
2. [Short reason that supports the claim, under 20 words]
3. [Short reason that supports the claim, under 20 words]

CONCLUSION:
[A dramatic, one-sentence summary reinforcing your false claim.]
"""
    # print(f"🚀 inside hallucinate() called with: {input_text}")
    response = client.invoke(prompt)
    # print(f"🪄 Gemini raw response:\n{response.content}")
    # print(f"🪄✅ Gemini raw response ended")
    return response.content.strip()
print(f"{datetime.now().strftime('%H:%M:%S')} ✅ Hallucinate logic ready")

# [4.3] Format GaslightBot Response

Gemini's raw output is parsed and cleaned using this function.

It extracts the three key components:
- **CLAIM** — The bold false statement  
- **LOGIC** — The three hallucinated justifications  
- **CONCLUSION** — The dramatic final summary

Extra whitespace is removed and only the first 3 logic lines are included (if more exist).

In [ ]:
def format_gaslight_response(response_text):
    """
    Formats Gemini's raw response into the official GaslightBot style.
    Handles structure enforcement and excess trimming.
    """
    import re
    # Clean and split response into parts using section headers
    claim_match = re.search(r"CLAIM:\s*(.*?)(?=\nLOGIC:)", response_text, re.DOTALL | re.IGNORECASE)
    logic_match = re.search(r"LOGIC:\s*(.*?)(?=\nCONCLUSION:)", response_text, re.DOTALL | re.IGNORECASE)
    conclusion_match = re.search(r"CONCLUSION:\s*(.*)", response_text, re.DOTALL | re.IGNORECASE)

    # Extract or default
    claim = claim_match.group(1).strip() if claim_match else "[Missing Claim]"
    logic_raw = logic_match.group(1).strip().split("\n") if logic_match else []
    conclusion = conclusion_match.group(1).strip() if conclusion_match else "[Missing Conclusion]"
    # Trim to max 3 list items
    logic_lines = [line.strip() for line in logic_raw if line.strip()]
    logic_lines = logic_lines[:3]

    # Format final output
    formatted = f"🤖 GaslightBot: {claim}\n"
    formatted += "---\n"
    formatted += "Here’s the breakdown:\n"
    for line in logic_lines:
        formatted += f"{line}\n"
    formatted += "---\n"
    formatted += f"{conclusion}\n"
    return formatted

print(f"{datetime.now().strftime('%H:%M:%S')} ✅ Answer Format Set")

### ❌[Unused] Gemini Model Setup❌
This line was originally intended to directly configure Gemini with LangChain,  
but has been moved inside specific LangGraph nodes like `hallucinate()` instead.

✅ You can remove or ignore this line.

In [ ]:
#llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
#print(f"{datetime.now().strftime('%H:%M:%S')} ✅ Gemini Model Set Up")

# [5.1] Create Human Node (User Input)

This node represents the human side of the conversation in LangGraph.

It:
- Displays the welcome message (only once)
- Accepts input if `RUN_INTERACTIVE` is enabled
- Checks for quit commands: `q`, `quit`, or `exit`
- Returns updated state with new `HumanMessage`

LangGraph will pass this state forward to the next node.

In [ ]:
from langchain_core.messages import HumanMessage

def human_node(state: GaslightState) -> GaslightState:
    if not state.get("messages"):
        print("🤖 GaslightBot: Welcome! Say something true, I dare you. I’ll fix it.")

    if state.get("finished"):
        # print(f"🧑✅ Check if state.get(finished) - Human Node. State-finish:", state["finished"])
        return state

    # Get user input
    if RUN_INTERACTIVE:
       #  print(f"🧑✅ RUN_INTERACTIVE is ON - State-finish:", state["finished"])
        user_input = input("🧠 You: ")
        print("User Input:", user_input)
    else:
        print(f"🧑❌ RUN_INTERACTIVE is OFF - State-finish:", state["finished"])
        user_input = "User input"

    # Check quit keywords
    if user_input.lower() in {"q", "quit", "exit"}:
        # print("🧑✅ Returning from Human_node with Exit key, State finished = True")
        return {
            "messages": state.get("messages", []),
            "finished": True
        }

    # Default: add message and continue
    # print("🧑 Returning from Human_node with user message")
    return {
        "messages": state.get("messages", []) + [HumanMessage(content=user_input)],
        "finished": False
    }

print(f"{datetime.now().strftime('%H:%M:%S')} ✅ Human Node Created")

# [5.2] Create Gaslight Node (AI Response Generator)

This is the core logic node of the chatbot.

It:
- Grabs the most recent `HumanMessage` from the state  
- Passes it into the `hallucinate()` function to get a false response from Gemini  
- Wraps the result as an `AIMessage`  
- Appends it to the message history  

LangGraph will then pass the updated state to the next node.

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

def gaslight_node(state: GaslightState) -> GaslightState:
    # print(f"💡📥 gaslight_node entered with State-finish:", state["finished"])
    # Get the latest user message
    last_user_msg = next(
        (m for m in reversed(state["messages"]) if isinstance(m, HumanMessage)),
        None
    )
    # Since we always start with Human Input, this should always be true
    assert last_user_msg is not None, "No HumanMessage found in state!"

    user_input = last_user_msg.content
    # print(f"💡🚀 hallucinate() called with: {user_input}")
    result = hallucinate(user_input)
    # print("💡✅ hallucinate() returned successfully, State:", state["finished"])
    new_output = AIMessage(content=result)
    # print("💡 Returning from gaslight_node, check new output", new_output)
    return {
        "messages": state["messages"] + [new_output],
        "finished": state.get("finished", False)
    }

print(f"{datetime.now().strftime('%H:%M:%S')} ✅ Chat Node Updated with hallucinate()")

# [6] Set Up LangGraph Flow

This section builds the node graph using LangGraph.

Steps:
- Create a new `StateGraph` using `GaslightState`
- Add both nodes: `human` and `gaslight`
- Set the entry point to `"human"`
- Define the flow:  
  → always go from `"human"` → `"gaslight"`

In this version, the conversation ends after one response.
You can modify `maybe_continue()` to loop until the user quits.

In [ ]:
from langgraph.graph import StateGraph, END

# Create the LangGraph
builder = StateGraph(GaslightState)

# Add your functional nodes
builder.add_node("human", human_node)
builder.add_node("gaslight", gaslight_node)

# Set entry point to start at the human node
builder.set_entry_point("human")

# Step 1: Always go from human → gaslight
builder.add_edge("human", "gaslight")

# Step 2: Conditionally return to human from gaslight, unless quitting
def maybe_continue(state: GaslightState):
#    return END if state.get("finished") else "human"
    return END  # 🚫 Always end the graph after gaslight_node

builder.add_conditional_edges("gaslight", maybe_continue)
# Compile the flow
graph = builder.compile()
print(f"{datetime.now().strftime('%H:%M:%S')} ✅ Flow Controller Set Up")

### ❌[Removed] Optional Loop Control Function❌  
This was an alternate `maybe_exit_human_node()` function designed to:
- Exit the conversation if `finished = True`
- Otherwise loop back to `"gaslight"`

In this simplified version of the project, the graph always ends after one cycle.<br>
You can uncomment this and wire it into LangGraph if you want **multi-turn interactions**.<br>
✅ For now you can remove or ignore this line.

In [ ]:
#from typing import Literal

#def maybe_exit_human_node(state: GaslightState) -> Literal["gaslight", "__end__"]:
#    return END if state.get("finished", False) else "gaslight"

#print(f"{datetime.now().strftime('%H:%M:%S')} ✅ loop Set Up")

### ❌[Unused] Old LangGraph Node Definition❌
This section wrapped the `hallucinate()` function as a LangGraph Runnable.  
It’s no longer needed because we directly use `hallucinate()` inside `gaslight_node`<br>
✅ Safe to delete or ignore.

In [ ]:
# Hallucinate Node
#from langchain_core.runnables import RunnableLambda

# Wrap hallucinate() as a runnable node for LangGraph
#hallucinate_node = RunnableLambda(
#    lambda state: state | {
#        "messages": [("assistant", hallucinate(state["messages"][-1][1]))]
#    }
#)

# [10] Run GaslightBot Loop

This loop starts the chatbot and continues until the user says `q`, `quit`, or `exit`.

Each cycle:
- Invokes the LangGraph (calls `graph.invoke(state)`)
- Checks if the user wants to quit
- Extracts the latest AI response
- Formats and prints it in GaslightBot’s confident, hallucinated

In [ ]:
print("💬 GaslightBot is now running (type `q`|`quit`|`exit` to quit)")
# print("✅ Starting loop with finished =", state["finished"])
while state["finished"] == False:
    # print("🔁 Calling graph.invoke, finished state =", state["finished"])
    state = graph.invoke(state)
    #print("✅📦 Full message state:")
    #for i, msg in enumerate(state["messages"], 1):
        # print(f"{i}. {msg.__class__.__name__.upper()}: {msg.content}")

     # 🔚 Only check for quit at end
    if state["finished"]:
        print("👋 Exiting GaslightBot. Goodbye, reality!")
        break

# 🧠 Get latest AI message
    latest_ai = next(
    (msg for msg in reversed(state["messages"]) if isinstance(msg, AIMessage)),None)

    if latest_ai:
        # print("🖨️ Final formatted result:")
        print(format_gaslight_response(latest_ai.content))
    else:
        print("⚠️ No assistant message found.")



In [ ]:
# config = {"recursion_limit": 50}
# Disabled the line below to prevent hanging while using Run-All
# state = gaslight_graph.invoke({"messages": [], "finished": False}, config)